In [1]:
!pip install numpy keras tensorflow scikit-learn scipy tqdm pillow opencv-python optuna
!pip install git+https://github.com/cair/tmu.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 61.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 70.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 86.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.6 MB/s eta 0:00:00:00:01
   ━━━━

In [8]:
!pip install git+https://github.com/cair/GraphTsetlinMachine.git

  Cloning https://github.com/cair/GraphTsetlinMachine.git to /tmp/pip-req-build-jp47ecmq
  Running command git clone --filter=blob:none --quiet https://github.com/cair/GraphTsetlinMachine.git /tmp/pip-req-build-jp47ecmq
  Resolved https://github.com/cair/GraphTsetlinMachine.git to commit f2041cf6919ac54b193e48cc97f04cbac7591ed1
  Preparing metadata (setup.py) ... done
  Created wheel for GraphTsetlinMachine: filename=GraphTsetlinMachine-0.2.0-py3-none-any.whl size=12944 sha256=be3229d6de4e14c5b3ccc4305f9410d282e473964a86268f373b66c92c7a76fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-low_e3qc/wheels/34/cc/e5/cd009a2fbcaf890e64951f3f7dc9df0cdece99f8c85b781178
Successfully built GraphTsetlinMachine


In [23]:
from GraphTsetlinMachine.graphs import Graphs
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
import numpy as np
from scipy.sparse import csr_matrix
from time import time
import argparse
from skimage.util import view_as_windows
from keras.datasets import mnist
from numba import jit

In [24]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = np.where(X_train > 75, 1, 0)
X_test = np.where(X_test > 75, 1, 0)
Y_train = Y_train.astype(np.uint32)
Y_test = Y_test.astype(np.uint32)


In [31]:
X_train.shape
Y_train.shape
X_train[0].shape
X_train[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 

In [32]:
def default_args(**kwargs):
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", default=250, type=int)
    parser.add_argument("--number-of-clauses", default=20000, type=int)
    parser.add_argument("--T", default=25000, type=int)
    parser.add_argument("--s", default=10.0, type=float)
    parser.add_argument("--depth", default=1, type=int)
    parser.add_argument("--hypervector-size", default=128, type=int)
    parser.add_argument("--hypervector-bits", default=2, type=int)
    parser.add_argument("--message-size", default=256, type=int)
    parser.add_argument("--message-bits", default=2, type=int)
    parser.add_argument("--max-included-literals", default=32, type=int)

    # This will ignore any unrecognized arguments (like the `-f` from Jupyter)
    args, unknown = parser.parse_known_args()

    for key, value in kwargs.items():
        if key in args.__dict__:
            setattr(args, key, value)
    
    return args

# Now you can call the args in the same way
args = default_args()


patch_size = 10
dim = 28 - patch_size + 1

number_of_nodes = dim * dim

symbol_names = []

# Column and row symbols
for i in range(dim):
    symbol_names.append("C:%d" % (i))
    symbol_names.append("R:%d" % (i))

# Patch pixel symbols
for i in range(patch_size*patch_size):
    symbol_names.append(i)
symbol_names

['C:0',
 'R:0',
 'C:1',
 'R:1',
 'C:2',
 'R:2',
 'C:3',
 'R:3',
 'C:4',
 'R:4',
 'C:5',
 'R:5',
 'C:6',
 'R:6',
 'C:7',
 'R:7',
 'C:8',
 'R:8',
 'C:9',
 'R:9',
 'C:10',
 'R:10',
 'C:11',
 'R:11',
 'C:12',
 'R:12',
 'C:13',
 'R:13',
 'C:14',
 'R:14',
 'C:15',
 'R:15',
 'C:16',
 'R:16',
 'C:17',
 'R:17',
 'C:18',
 'R:18',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [ ]:
graphs_train = Graphs(X_train.shape[0], symbol_names=symbol_names, hypervector_size=args.hypervector_size, hypervector_bits=args.hypervector_bits)
for graph_id in range(X_train.shape[0]):
    graphs_train.set_number_of_graph_nodes(graph_id, number_of_nodes)

graphs_train.prepare_node_configuration()

for graph_id in range(X_train.shape[0]):
    for node_id in range(graphs_train.number_of_graph_nodes[graph_id]):
        graphs_train.add_graph_node(graph_id, node_id, 0)

graphs_train.prepare_edge_configuration()

for graph_id in range(X_train.shape[0]):
    if graph_id % 1000 == 0:
        print(graph_id, X_train.shape[0])
     
    windows = view_as_windows(X_train[graph_id,:,:], (patch_size, patch_size))
    for q in range(windows.shape[0]):
            for r in range(windows.shape[1]):
                node_id = q*dim + r

                patch = windows[q,r].reshape(-1).astype(np.uint32)
                for k in patch.nonzero()[0]:
                    graphs_train.add_graph_node_feature(graph_id, node_id, k)

                graphs_train.add_graph_node_feature(graph_id, node_id, "C:%d" % (q))
                graphs_train.add_graph_node_feature(graph_id, node_id, "R:%d" % (r))

graphs_train.encode()

print("Training data produced")

graphs_test = Graphs(X_test.shape[0], init_with=graphs_train)
for graph_id in range(X_test.shape[0]):
    graphs_test.set_number_of_graph_nodes(graph_id, number_of_nodes)

graphs_test.prepare_node_configuration()

for graph_id in range(X_test.shape[0]):
    for node_id in range(graphs_test.number_of_graph_nodes[graph_id]):
        graphs_test.add_graph_node(graph_id, node_id, 0)

graphs_test.prepare_edge_configuration()

for graph_id in range(X_test.shape[0]):
    if graph_id % 1000 == 0:
        print(graph_id, X_test.shape[0])
     
    windows = view_as_windows(X_test[graph_id,:,:], (10, 10))
    for q in range(windows.shape[0]):
            for r in range(windows.shape[1]):
                node_id = q*dim + r

                patch = windows[q,r].reshape(-1).astype(np.uint32)
                for k in patch.nonzero()[0]:
                    graphs_test.add_graph_node_feature(graph_id, node_id, k)

                graphs_test.add_graph_node_feature(graph_id, node_id, "C:%d" % (q))
                graphs_test.add_graph_node_feature(graph_id, node_id, "R:%d" % (r))

graphs_test.encode()

print("Testing data produced")

In [21]:
tm = MultiClassGraphTsetlinMachine(args.number_of_clauses, args.T, args.s, depth=args.depth, message_size=args.message_size, message_bits=args.message_bits, max_included_literals=args.max_included_literals)

Initialization of sparse structure.


In [22]:
for i in range(args.epochs):
    start_training = time()
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
    stop_training = time()

    start_testing = time()
    result_test = 100*(tm.predict(graphs_test) == Y_test).mean()
    stop_testing = time()

    result_train = 100*(tm.predict(graphs_train) == Y_train).mean()

    print("%d %.2f %.2f %.2f %.2f" % (i, result_train, result_test, stop_training-start_training, stop_testing-start_testing))

weights = tm.get_state()[1].reshape(2, -1)
for i in range(tm.number_of_clauses):
        print("Clause #%d W:(%d %d)" % (i, weights[0,i], weights[1,i]), end=' ')
        l = []
        for k in range(args.hypervector_size * 2):
            if tm.ta_action(0, i, k):
                if k < args.hypervector_size:
                    l.append("x%d" % (k))
                else:
                    l.append("NOT x%d" % (k - args.hypervector_size))
        print(" AND ".join(l))


start_training = time()
tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
stop_training = time()

start_testing = time()
result_test = 100*(tm.predict(graphs_test) == Y_test).mean()
stop_testing = time()

result_train = 100*(tm.predict(graphs_train) == Y_train).mean()

print("%.2f %.2f %.2f %.2f" % (result_train, result_test, stop_training-start_training, stop_testing-start_testing))

print(graphs_train.hypervectors)

0 13.04 13.50 44.75 3.08
1 13.37 12.93 38.77 3.10
2 13.46 13.22 37.89 3.09
3 13.33 13.03 37.51 3.09
4 13.28 13.32 37.28 3.09
5 13.48 13.07 37.13 3.09
6 12.93 12.61 37.02 3.09
7 13.18 13.52 36.92 3.07
8 13.31 13.37 36.89 3.09
9 13.54 13.44 36.78 3.09
10 13.44 13.32 36.69 3.09
11 13.40 13.80 36.68 3.09
12 13.15 13.11 36.63 3.09
13 13.40 13.75 36.59 3.07
14 13.43 12.82 36.60 3.09
15 13.03 13.48 36.56 3.09
16 13.10 13.17 36.57 3.07
17 13.48 13.17 36.51 3.09
18 13.35 13.25 36.48 3.09
19 13.13 13.65 36.39 3.07
20 13.18 13.15 36.43 3.09
22 13.46 12.78 36.42 3.09
23 13.38 13.10 36.35 3.09
24 13.50 13.63 36.45 3.09
25 13.11 13.26 36.35 3.09
26 13.30 13.09 36.32 3.09
27 13.23 13.47 36.35 3.09
28 13.25 13.71 36.32 3.09
29 13.49 13.86 36.35 3.09
30 13.51 12.89 36.30 3.09
31 13.36 12.53 36.31 3.09
32 13.69 14.03 36.31 3.09
33 13.26 12.85 36.35 3.09
34 13.28 13.51 36.27 3.09
36 13.29 13.13 36.25 3.08
37 13.63 13.81 36.29 3.09
38 13.34 13.14 36.22 3.08
39 13.51 12.72 36.23 3.09
40 13.47 12.74 36.28 3

KeyboardInterrupt: 